# 1. Initialisation

In [ ]:
!pip install sentence_transformers > /dev/null
from sentence_transformers import SentenceTransformer

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [ ]:
import nltk
import numpy as np
from string import punctuation
import re
from keras.preprocessing import text
import pandas as pd
nltk.download('stopwords')
from keras.preprocessing.sequence import skipgrams
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
from torch.optim import Adam, RMSprop
nltk.download('punkt')
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import re
from sklearn.utils import shuffle
import time
from datetime import datetime
import os
from tqdm.auto import tqdm
import json
from google.colab import drive
from torch.utils.data import DataLoader,TensorDataset

device = torch.device('cuda')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# 2. Getting data from question bank

In [ ]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1RHA3WUqdXuJruEIxS1spgjRhUnmTujVml83D1wMt_HI')
question_bank = pd.DataFrame(wb.worksheet('Question_Bank').get_all_values())

question_bank.columns = question_bank.iloc[0,:]
question_bank = question_bank.iloc[1:,:]

In [ ]:
# Please don't remove this
question_bank['Answer'] = question_bank.Answer.apply(lambda x: x if x is not np.nan else '')
question_bank['Answer_2'] = question_bank.Answer_2.apply(lambda x: x if x is not np.nan else '')
question_bank['Answer_3'] = question_bank.Answer_3.apply(lambda x: x if x is not np.nan else '')

In [ ]:
bert = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

#3. Imports and installs

In [ ]:
!pip install pymysql
import pymysql

     |████████████████████████████████| 43 kB 1.5 MB/s 


In [ ]:
!pip install mysqlclient
import MySQLdb
import pandas as pd
import numpy as np
import re


     |████████████████████████████████| 88 kB 3.3 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.0.3-cp37-cp37m-linux_x86_64.whl size=100147 sha256=aab1f81222b2f74f397efe620c5a2b688061403e542af1b2d3f3222c9f469ac1
  Stored in directory: /root/.cache/pip/wheels/79/1c/f8/11fafab45fe6696eea63794a5d747b9c6b54990ac6f1885fb7
Successfully built mysqlclient


#4. Getting data from google sheet

In [ ]:
from google.colab import auth
from copy import deepcopy
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1RHA3WUqdXuJruEIxS1spgjRhUnmTujVml83D1wMt_HI')
sheet = wb.worksheet('Question_Bank')
data = sheet.get_all_values()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:]

In [ ]:
%load_ext google.colab.data_table

In [ ]:
df.reset_index(drop=True,inplace=True) 
df.head(2)

,Unique_Question_ID,Category,Question,Answer,Difficulty level,MaxMarks,Answer_2,Answer_3,User Answer (self)
0,0,SVM,How does SVM map non linear boundaries?,By using the kernel trick,1,5,Using the kernel function,Through the kernel method,
1,1,SVM,What are the different kernels that you know i...,Linear Polynomial Sigmoid RBF,1,5,"Radial basis function, euclidean, gaussian","Radial basis function, euclidean, gaussian",


In [ ]:
import re

In [ ]:
QBAD_SYMBOLS_RE = re.compile("""[/^'"#\\\\|\-`~_]""")

In [ ]:
def removeBadChar(x):
    x = str(x)
    x = re.sub(pattern=QBAD_SYMBOLS_RE,repl='', string = x)
    return x


In [ ]:
df['Question'] = df.Question.apply(lambda x: removeBadChar(x))

#5. Preparing the database instance for CRUD ops

In [ ]:
hostName = 'db4free.net'      
userName = 'udreedbczu'          
passWord = 'ezcb9vazqz'           
dbName =  userName                
DBConn= MySQLdb.connect(hostName,userName,passWord,dbName)
def runCMD (DDL):
    
    """

    MySQL function for CUD of CRUD

    """

    DBConn= MySQLdb.connect(hostName,userName,passWord,dbName)
    myCursor = DBConn.cursor()
    retcode = myCursor.execute(DDL) 
    print (retcode)
    DBConn.commit()
    DBConn.close()

def runSELECT (CMD):
    
    """

    MySQL function for R of CRUD

    """

    DBConn= MySQLdb.connect(hostName,userName,passWord,dbName)
    df_mysql = pd.read_sql(CMD, con=DBConn)    
    DBConn.close()
    return df_mysql

def r(msg):
    
    """

    This function automatically routes all SQL operations to runCMD or runSELECT automatically
    
    """
    
    if msg[0:6]=="SELECT" or msg[0:6]=="select":
        return runSELECT(msg)
    else:
        runCMD(msg)

#6. Preparing the dataset of each user

In [ ]:
# r('delete from userQA where userid ="shahrukh.gouhar@praxis.ac.in"')

255


In [ ]:
# r('select question, answer, saved_date, userid, row_number() over (partition by userid, CAST(saved_date as DATE), question order by saved_date DESC) as rn\
#  from userQA')

In [ ]:
# Fetching user answer data for today's date ~~~~~ ~~~~~ 

userInfoFromDB = r('select question, answer,  saved_date, userid from (\
 select question, answer, saved_date, userid, row_number() over (partition by userid, CAST(saved_date as DATE), question order by saved_date DESC) as rn\
 from userQA\
 where CAST(saved_date AS DATE)= CAST("2021-11-13" AS Date)\
 ) t2\
 where rn = 1')

#  where CAST(saved_date AS DATE)= CAST(NOW() AS Date)-15\

# DO CAST(NOW() AS Date)-1 -->  FOR YESTERDAY'S SCORES

In [ ]:
userInfoFromDB

,question,answer,saved_date,userid
0,Do outliers affect the SVMmodel?,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in
1,Does the margin need to be maximum or minimum?,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in
2,How to identify support vectors?,lagrange multiplier will be zero.,2021-11-12 18:56:40.098387,radha.rathore@praxis.ac.in
3,Is SVM a deterministic or non deterministic al...,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in
4,Name 7 different solvers implemented in sklear...,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in
5,What does BLUE property of estimators stands f...,,2021-11-12 18:54:46.407180,radha.rathore@praxis.ac.in
6,What is assumption of Homoskedasticity in line...,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in
7,What is assumption of Independence in linear r...,,2021-11-12 18:54:42.922288,radha.rathore@praxis.ac.in
8,What is C in SVM?,hyperparameter,2021-11-12 18:55:55.330381,radha.rathore@praxis.ac.in
9,What is the difference between cost function o...,LASSO uses L1 regularization which is sum of a...,2021-11-12 18:54:31.036868,radha.rathore@praxis.ac.in


In [ ]:
ansTable = pd.merge(userInfoFromDB, df, left_on='question', right_on='Question')  # Merging the user answer data with the master answer data

In [ ]:
ansTable

,question,answer,saved_date,userid,Unique_Question_ID,Category,Question,Answer,Difficulty level,MaxMarks,Answer_2,Answer_3,User Answer (self)
0,Do outliers affect the SVMmodel?,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in,9,SVM,Do outliers affect the SVMmodel?,No,2,10,No,No,
1,Does the margin need to be maximum or minimum?,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in,5,SVM,Does the margin need to be maximum or minimum?,Maximum,1,5,Max,Max,
2,How to identify support vectors?,lagrange multiplier will be zero.,2021-11-12 18:56:40.098387,radha.rathore@praxis.ac.in,16,SVM,How to identify support vectors?,Lagrange multipliers will be greater than zero,3,15,Alpha will be positive,Alpha will be positive,
3,Is SVM a deterministic or non deterministic al...,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in,14,SVM,Is SVM a deterministic or non deterministic al...,Deterministic,2,10,Deterministic,Deterministic,
4,Name 7 different solvers implemented in sklear...,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in,80,Linear Regression,Name 7 different solvers implemented in sklear...,"auto, svd, cholesky, lsqr, sparse cg, sag, saga",3,15,"auto, svd, cholesky, lsqr, sparse cg, sag, saga","auto, svd, cholesky, lsqr, sparse cg, sag, saga",
5,What does BLUE property of estimators stands f...,,2021-11-12 18:54:46.407180,radha.rathore@praxis.ac.in,67,Linear Regression,What does BLUE property of estimators stands f...,"Unbiasedness, Constant Variance, Linearity",2,10,"Unbiasedness, Constant Variance, Linearity","Unbiasedness, Constant Variance, Linearity",
6,What is assumption of Homoskedasticity in line...,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in,50,Linear Regression,What is assumption of Homoskedasticity in line...,The variance of error terms are similar across...,2,10,The error terms shoud have constant variance a...,The variance of errors should not vary across ...,
7,What is assumption of Independence in linear r...,,2021-11-12 18:54:42.922288,radha.rathore@praxis.ac.in,34,Linear Regression,What is assumption of Independence in linear r...,Observation are independent of each other.,1,5,Each sample is independent of every other sample.,Each Observation is independent of every other...,
8,What is C in SVM?,hyperparameter,2021-11-12 18:55:55.330381,radha.rathore@praxis.ac.in,2,SVM,What is C in SVM?,A model hyperparameter which is the regularisa...,1,5,regularisation coefficient for the misclassifi...,regularisation coefficient for the misclassifi...,
9,What is the difference between cost function o...,LASSO uses L1 regularization which is sum of a...,2021-11-12 18:54:31.036868,radha.rathore@praxis.ac.in,38,Linear Regression,What is the difference between cost function o...,LASSO uses L1 regularization which is sum of a...,1,5,In LASSO cost function is sum of mean square e...,In LASSO cost function is sum of mean square e...,


In [ ]:
# ansTable.drop(columns=['Unique_Question_ID', 'Question', 'Difficulty level'], axis=1, inplace=True)  # removing few columns form merged data

ansTable.drop(columns=['Unique_Question_ID', 'Question', 'Difficulty level', 'User Answer (self)'], axis=1, inplace=True)  # removing few columns form merged data

In [ ]:
# ansTable.head(2)   #Merged Dataframe 

In [ ]:
# len(ansTable)

In [ ]:
# ansTable

## Single answers handling 

In [ ]:
# type(ansTable)

In [ ]:
ansTable.rename({'answer': 'User_answer'}, axis = 1, inplace=True) 

In [ ]:
# ansTable

In [ ]:
le=len(ansTable)
le

18

In [ ]:
ansTable['Answer']

0                                                    No
1                                               Maximum
2        Lagrange multipliers will be greater than zero
3                                         Deterministic
4       auto, svd, cholesky, lsqr, sparse cg, sag, saga
5            Unbiasedness, Constant Variance, Linearity
6     The variance of error terms are similar across...
7           Observation  are independent of each other.
8     A model hyperparameter which is the regularisa...
9     LASSO uses L1 regularization which is sum of a...
10                  More misclassification would happen
11                                    Mean Square Error
12                                                  Yes
13                                         1084 to 2196
14                                                 More
15                               It will be very large.
16                                        Deterministic
17    Depending upon whether the penalty incurre

In [ ]:
sing_answerT=deepcopy(ansTable)
multi_answerT=deepcopy(ansTable)

Checking for Answer length (taken from question bank) if it is 1 then Single answer else it is a lengthy answer

In [ ]:
for i in range(le):
    # print(ansTable['Answer'][i])
    # print(re.findall(r'\w+',ansTable['Answer'][i]))
    # print(re.findall(r'\w+',  "dd_ , dd@ , dd%, dd(), dd/, dd's,  ff, \77 ,/7 ;dss dfsf \6"))
    # break
    res=len(re.findall(r'\w+',ansTable['Answer'][i]))
    if res==1:
        multi_answerT.drop(i,inplace=True, axis=0)
    else:
        sing_answerT.drop(i,inplace=True, axis=0)


sing_answerT.reset_index(drop=True, inplace=True)
multi_answerT.reset_index(drop=True, inplace=True)

Dividing single and Multipe answers into two dataframes

In [ ]:
# sing_answerT

In [ ]:
# multi_answerT

In [ ]:
# multi_answerT.to_csv('multianswer.csv')

creating a list of scoring for each answer and appending it to the single answer dataframe 

In [ ]:
# Need to compare more carefully
le_sing=len(sing_answerT)
sing_score=[]
for i in range(le_sing):
    if sing_answerT['User_answer'][i].lower().split(' ')[0].strip(' ')==sing_answerT['Answer'][i].lower().strip(" "):
        sing_score.append(int(sing_answerT['MaxMarks'][i]))
    elif sing_answerT['User_answer'][i].lower().split(' ')[0].strip(' ')==sing_answerT['Answer_2'][i].lower().strip(" "):
        sing_score.append(int(sing_answerT['MaxMarks'][i]))
    elif sing_answerT['User_answer'][i].lower().split(' ')[0].strip(' ')==sing_answerT['Answer_3'][i].lower().strip(" "):
        sing_score.append(int(sing_answerT['MaxMarks'][i]))
    else:
        sing_score.append(0)
print(sing_score)

[0, 0, 0, 10, 5, 5]


In [ ]:
sing_answerT['Scores_Obtained']=sing_score

In [ ]:
# sing_answerT

#7. Scoring the user

# 7. Loading tester and skipgram model 

In [ ]:
class tester(nn.Module):

    def __init__(self, hidden_size, num_layers,e_layer ,embedding_dim, bidir=False):

        super(tester, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_dim = embedding_dim
        self.bidir=bidir
        self.num_layers = num_layers

        self.embed = emb_layer

        self.gru1 = nn.GRU(input_size=self.embedding_dim, 
                            hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, 
                            batch_first=True,
                            bidirectional = self.bidir,
                            dropout=0.0)
        self.gru2 = nn.GRU(input_size=self.embedding_dim, 
                            hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, 
                            batch_first=True,
                            bidirectional = self.bidir,
                            dropout=0.0)
        

    def forward(self, x, y,hx,hy):
        x = self.embed(x.long()) 
        y = self.embed(y.long())


        x, hx = self.gru1(x,hx)
        y, hy = self.gru2(y,hy)

        x = x[:,-1]  #out[:,-1]
        y = y[:,-1]


        x = F.leaky_relu(x)
        y = F.leaky_relu(y)

        out = F.cosine_similarity(x,y)
        return out,hx,hy


    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.num_layers, batch_size, self.hidden_size).zero_().to(device)
        return hidden



class skipgram(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim=64):
        super(skipgram, self).__init__()

        self.u_embeddings = nn.Embedding(vocab_size, embedding_dim,sparse=False)   


    def forward(self, u_pos, v_pos ):

        embed_u = self.u_embeddings(u_pos.long())
        embed_v = self.u_embeddings(v_pos.long())


        return embed_u, embed_v

In [ ]:
!gdown https://drive.google.com/uc?id=1590y0Q1LNEibDFy7Wx0cqXPtInjtv8yH
!gdown https://drive.google.com/uc?id=1ABIJgdJSJO1FK-ioPame18mYqTq5hM4C
!gdown https://drive.google.com/uc?id=1deqPKvQLSMGeKVfHif7vsb4ToxStTu3v
!gdown https://drive.google.com/uc?id=1yYGo_99XwfiIiTNpHIDhXlG54GHCZL40

Downloading...
From: https://drive.google.com/uc?id=1590y0Q1LNEibDFy7Wx0cqXPtInjtv8yH
To: /content/embedding_network.pt
100% 1.97M/1.97M [00:00<00:00, 59.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ABIJgdJSJO1FK-ioPame18mYqTq5hM4C
To: /content/rf.pkl
100% 985k/985k [00:00<00:00, 65.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1deqPKvQLSMGeKVfHif7vsb4ToxStTu3v
To: /content/word2id.json
100% 138k/138k [00:00<00:00, 51.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yYGo_99XwfiIiTNpHIDhXlG54GHCZL40
To: /content/final_tester.pt
100% 2.24M/2.24M [00:00<00:00, 65.0MB/s]


In [ ]:
"""
Creating an instance of the skipgram class and loading the saved state_dict

"""



######### LOADING WORD2ID FROM JSON ################
# Opening JSON file
# Path need to be changed later
with open('word2id.json','r') as json_file:
    word2id = json.load(json_file)
  
    # Print the type of data variable
    print("Type:", type(word2id))

####################################################

embedding_dim = 64
vocab_size = 7709
embeddings = skipgram(vocab_size)
embeddings = embeddings.to(device)

if True:  # When you train word embedding above then make this true
    embeddings.load_state_dict(torch.load('embedding_network.pt'))
    embeddings = embeddings.to(device)
    print('model loaded')
for para in embeddings.parameters():
    para.requires_grad = False

emb_layer = embeddings.u_embeddings
# emb_layer = embeddings.v_embeddings

Type: <class 'dict'>
model loaded


In [ ]:
"""
Creating an instance of the tester class and loading the saved state_dict

"""

## Also provide embedding layer i.e. e_layer=emb_layer
nnModel = tester(hidden_size=50,num_layers=2,e_layer=emb_layer,embedding_dim=64)

In [ ]:
path = 'final_tester.pt'

if True:  # When you train word embedding above then make this true
    nnModel.load_state_dict(torch.load(path)) # path need to be changed
    nnModel = nnModel.to(device)
    print('model loaded')

for para in nnModel.parameters():
    para.requires_grad = False

model loaded


## Machine learning model

In [ ]:
# !gdown https://drive.google.com/uc?id=1YADTDb21RDbsLTQbwqNFl3_CpSwb4bg2
with open('rf.pkl','rb') as file:
    rf_model = pickle.load(file)

# PREDICTION USING NN MODEL

In [ ]:
def coder(sentence, max_len):
    BAD_SYMBOLS_RE = re.compile("""[/$%{}^'"#\\\\|@,;[\]\-`~_]""")
    sent=[]
    try:
        sentence = re.sub('=','equals', sentence)
        sentence = re.sub(pattern=BAD_SYMBOLS_RE,repl=' ', string = sentence)
        sentence = re.sub('[^\w+]',' ', sentence)

        words = sentence.lower().split()
        
        for word in words:
            try:
                sent.append(word2id[word])
            except:
                pass
        
    except:
        pass
    
        
    x = torch.from_numpy(np.array(sent))
    pad_size = list(x.shape)
    pad_size[0] = max_len - x.size(0)
    x = torch.cat([x, torch.zeros(*pad_size)], dim=0)
    x = x[0:max_len]
    return x

In [ ]:
coder('',50)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], dtype=torch.float64)

In [ ]:
coder('cccdddff',50)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], dtype=torch.float64)

In [ ]:
coder('',50) == coder('linear',50)

tensor([False,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True])

In [ ]:
coder('',50) == coder('lineardfdsfsd  dsdfdf sdsf re',50)

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True])

In [ ]:
us = np.array([coder(' dssdf fsdffd', 50).numpy()])
us.sum() == 0.0

True

In [ ]:
us = np.array([coder(' dssdf fsdffd linear', 50).numpy()])
us.sum() == 0.0

False

In [ ]:
us = np.array([coder(' ', 50).numpy()])
us.sum() == 0.0

True

#8. SCORER

In [ ]:
# fthx = nnModel.init_hidden(batch_size=len(actuals_coded_t))
# fthy = nnModel.init_hidden(batch_size=len(actuals_coded_t))

# nnPred,thx,thy = nnModel(actuals_coded_t, variations_coded_t,fthx,fthy)
# nnPred = nnPred.detach().cpu().numpy()

In [ ]:
def scorer(a,b,c,d):
    
    """
    Inputs:

    a = list of User answer
    b = list of Actual answer 1
    c = list of Actual answer 2
    d = list of Actual answer 3    

    Processing:

    BERT_dist: Maximum similarity to answers as measured by BERT ( global information )
    LSA_dist: Maximum similarity to answers as measured by LSA technique ( local information )

    We are feeding these distances to a trained 'model' which returns the final score

    Outputs:

    Score obtained: ML(Count_dist, BERT_dist)

    """
    
    score_list =[]

    for i in range(len(a)):
      
      #BERT
        if a[i] != '':
            bert_a = bert.encode([a[i]]).reshape(1,-1)
            bert_b = bert.encode([b[i]]).reshape(1,-1)
            bert_c = bert.encode([c[i]]).reshape(1,-1)
            bert_d = bert.encode([d[i]]).reshape(1,-1)

            BERT_dist = max(cosine_similarity(bert_a, bert_b)[0][0], 
                            cosine_similarity(bert_a, bert_c)[0][0],
                            cosine_similarity(bert_a, bert_d)[0][0]) 

            ######################################
            #NEURAL NET
            max_len = 50
            user_answer = np.array([coder(a[i], max_len).numpy()])
            if user_answer.sum() != 0.0:
            
                answer1 = np.array([coder(b[i], max_len).numpy()])
                answer2 = np.array([coder(c[i], max_len).numpy()])
                answer3 = np.array([coder(d[i], max_len).numpy()])

                user_answer_t = torch.from_numpy(user_answer).to(device)
                answer1_t = torch.from_numpy(answer1).to(device)
                answer2_t = torch.from_numpy(answer2).to(device)
                answer3_t = torch.from_numpy(answer3).to(device)

                fthx = nnModel.init_hidden(batch_size=len(user_answer_t))
                fthy1 = nnModel.init_hidden(batch_size=len(answer1_t))
                fthy2 = nnModel.init_hidden(batch_size=len(answer1_t))
                fthy3 = nnModel.init_hidden(batch_size=len(answer1_t))
                nnPred1,_,_ = nnModel(user_answer_t, answer1_t,fthx,fthy1)
                nnPred2,_,_ = nnModel(user_answer_t, answer2_t, fthx,fthy2)
                nnPred3,_,_ = nnModel(user_answer_t, answer3_t, fthx,fthy3)
                
                nnPred1 = nnPred1.detach().cpu().numpy()
                nnPred2 = nnPred2.detach().cpu().numpy()
                nnPred3 = nnPred3.detach().cpu().numpy()


                nn_dist = np.max([nnPred1, nnPred2, nnPred3])
            else:
                nn_dist = 0
            
            # print(f'BERT : {BERT_dist}')
            # print(f'NN : {nn_dist}')

            ######################               


                            
            score = rf_model.predict([[BERT_dist, nn_dist]])

            # print(f'Score : {score[0]}')
            # print('***********************')
            score_list.append(score[0])
        else:
            score_list.append(0)

    return score_list

## ~~ RUN

In [ ]:
score_List = scorer(list(multi_answerT.User_answer.apply(lambda x:x if x is not np.nan else '')),
                    list(multi_answerT.Answer.apply(lambda x:x if x is not np.nan else '')), 
                    list(multi_answerT.Answer_2.apply(lambda x:x if x is not np.nan else '')), 
                    list(multi_answerT.Answer_3.apply(lambda x:x if x is not np.nan else '')))  # sending the user and standard answers for scoring

In [ ]:
# score_List

In [ ]:
# len(score_List)

In [ ]:
ansScores_df = deepcopy(multi_answerT)

In [ ]:
def score_threshold(sc):

    if sc<=0.5:
        m = 0
    else:
        sc = sc-0.5

        if 0<sc<=0.4:
            m = sc + 0.6
        else:
            m = 1
    return m

In [ ]:
score_List2 = [score_threshold(x) for x in score_List]

In [ ]:
ansScores_df["Scores_Obtained"] = score_List2 * ansScores_df.MaxMarks.astype('float')  # multiplying similarity score with maximum marks to get final score

In [ ]:
ansScores_df.shape,sing_answerT.shape

((12, 10), (6, 10))

In [ ]:
ansScores_df = pd.concat([ansScores_df,sing_answerT]).reset_index(drop=True)

In [ ]:
# ansScores_df

In [ ]:
ansScores_df["saved_date"] = pd.to_datetime(ansScores_df["saved_date"])  #converting saved_date from string to date

In [ ]:
ansScores_df["MaxMarks"] = ansScores_df["MaxMarks"].astype('int')  # converting MaxMarks from string to int

In [ ]:
# ansScores_df.head(2)   # Dataframe with scores generated for each question

In [ ]:
# len(ansScores_df)

In [ ]:
ansScores_df

,question,User_answer,saved_date,userid,Category,Answer,MaxMarks,Answer_2,Answer_3,Scores_Obtained
0,How to identify support vectors?,lagrange multiplier will be zero.,2021-11-12 18:56:40.098387,radha.rathore@praxis.ac.in,SVM,Lagrange multipliers will be greater than zero,15,Alpha will be positive,Alpha will be positive,0.000
1,Name 7 different solvers implemented in sklear...,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in,Linear Regression,"auto, svd, cholesky, lsqr, sparse cg, sag, saga",15,"auto, svd, cholesky, lsqr, sparse cg, sag, saga","auto, svd, cholesky, lsqr, sparse cg, sag, saga",0.000
2,What does BLUE property of estimators stands f...,,2021-11-12 18:54:46.407180,radha.rathore@praxis.ac.in,Linear Regression,"Unbiasedness, Constant Variance, Linearity",10,"Unbiasedness, Constant Variance, Linearity","Unbiasedness, Constant Variance, Linearity",0.000
3,What is assumption of Homoskedasticity in line...,,2021-11-12 18:48:36.490895,radha.rathore@praxis.ac.in,Linear Regression,The variance of error terms are similar across...,10,The error terms shoud have constant variance a...,The variance of errors should not vary across ...,0.000
4,What is assumption of Independence in linear r...,,2021-11-12 18:54:42.922288,radha.rathore@praxis.ac.in,Linear Regression,Observation are independent of each other.,5,Each sample is independent of every other sample.,Each Observation is independent of every other...,0.000
5,What is C in SVM?,hyperparameter,2021-11-12 18:55:55.330381,radha.rathore@praxis.ac.in,SVM,A model hyperparameter which is the regularisa...,5,regularisation coefficient for the misclassifi...,regularisation coefficient for the misclassifi...,5.000
6,What is the difference between cost function o...,LASSO uses L1 regularization which is sum of a...,2021-11-12 18:54:31.036868,radha.rathore@praxis.ac.in,Linear Regression,LASSO uses L1 regularization which is sum of a...,5,In LASSO cost function is sum of mean square e...,In LASSO cost function is sum of mean square e...,5.000
7,What would happen when you use a very low valu...,more misclassification,2021-11-12 18:55:48.099088,radha.rathore@praxis.ac.in,SVM,More misclassification would happen,5,More points would be wrongly classified,More observations will be misclassified,4.925
8,Which performance metric is used as cost funct...,mean square error,2021-11-12 18:54:40.385717,radha.rathore@praxis.ac.in,Linear Regression,Mean Square Error,5,MSE,MSE,3.300
9,"If 2000 is value of an estimator, 100 is stand...",,2021-11-12 18:47:20.610880,shahrukh.gouhar@praxis.ac.in,Linear Regression,1084 to 2196,10,1084 to 2196,1084 to 2196,0.000


#9. Adding user details to a new table to be fetched by the client later

In [ ]:
# Creating 'testScores' table if doesn't exist to score the scores for each answer

# SAVE IN OTHER FILE

r('CREATE TABLE IF NOT EXISTS testScores\
 (question longtext,\
 User_answer longtext,\
 saved_date datetime,\
 userid text,\
 Category text,\
 Answer longtext,\
 MaxMarks int,\
 Answer_2 longtext,\
 Answer_3 longtext,\
 Scores_Obtained float\
 )')

0


In [ ]:
# Storing the scores calculated for each answer in the 'testScores' table

connection = pymysql.connect(host=hostName,
                             user=userName,
                             password=passWord,
                             db=dbName)
cursor=connection.cursor()

for index, row in ansScores_df.iterrows():
  sql = "INSERT INTO testScores (question,User_answer, saved_date, userid, Category, Answer, MaxMarks, Answer_2, Answer_3, Scores_Obtained) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

  cursor.execute(sql, (row.question, row.User_answer, row.saved_date, row.userid, row.Category, row.Answer, row.MaxMarks, row.Answer_2, row.Answer_3, row.Scores_Obtained))

connection.commit()
connection.close()

In [ ]:
# datad= data_n[['saved_date', 'userid', 'question', 'User_answer',  'Answer',
#        'Answer_2', 'Answer_3', 'Scores_Obtained', 'MaxMarks','Category']]

#datad

In [ ]:
# datad.to_csv("demo_test_07-10.csv", index=False)

# 10. Categorywise score

In [ ]:
scores_df_2 = deepcopy(ansScores_df)

In [ ]:
scores_df_2.saved_date = scores_df_2.saved_date.dt.date

In [ ]:
# scores_df_2.head()

In [ ]:
categorywise_scores = scores_df_2[['userid', 'saved_date', 'Category', 'Scores_Obtained', 'MaxMarks' ]].groupby(['userid', 'saved_date', 'Category']).sum()

In [ ]:
# categorywise_scores

In [ ]:
categorywise_scores.reset_index(inplace=True)  

In [ ]:
# categorywise_scores

In [ ]:
r('CREATE TABLE IF NOT EXISTS categorywise_scores\
 (userid text,\
 saved_date date,\
 Category text,\
 Scores_Obtained float,\
 MaxMarks int\
 )')

0


In [ ]:
connection = pymysql.connect(host=hostName,
                             user=userName,
                             password=passWord,
                             db=dbName)
cursor=connection.cursor()

for index, row in categorywise_scores.iterrows():
  sql = "INSERT INTO categorywise_scores (userid, saved_date, Category, Scores_Obtained, MaxMarks) VALUES (%s, %s, %s, %s, %s)"

  cursor.execute(sql, (row.userid, row.saved_date, row.Category, row.Scores_Obtained, row.MaxMarks))

connection.commit()
connection.close()

In [ ]:
r('select * from categorywise_scores')

,userid,saved_date,Category,Scores_Obtained,MaxMarks
0,radha.rathore@praxis.ac.in,2021-11-12,Linear Regression,8.300,50
1,radha.rathore@praxis.ac.in,2021-11-12,SVM,9.925,50
2,shahrukh.gouhar@praxis.ac.in,2021-11-12,Linear Regression,25.000,50


#11. OVERALL SCORE

In [ ]:
overall_scores = r('select * from categorywise_scores')
overall_scores

,userid,saved_date,Category,Scores_Obtained,MaxMarks
0,radha.rathore@praxis.ac.in,2021-11-12,Linear Regression,8.300,50
1,radha.rathore@praxis.ac.in,2021-11-12,SVM,9.925,50
2,shahrukh.gouhar@praxis.ac.in,2021-11-12,Linear Regression,25.000,50


In [ ]:
overall_scores['saved_date'] = overall_scores.saved_date.astype(np.datetime64)

In [ ]:
# overall_scores

In [ ]:
overall_scores['Category'] = overall_scores.groupby(['userid', 'saved_date'])['Category'].transform(lambda x : '; '.join(x))

In [ ]:
overall_scores = overall_scores.groupby(['userid', 'saved_date', 'Category']).sum()

In [ ]:
# overall_scores

In [ ]:
overall_scores.reset_index(inplace=True)  

In [ ]:
# overall_scores

In [ ]:
r('CREATE TABLE IF NOT EXISTS overall_Scores\
 (userid text,\
 saved_date date,\
 Category text,\
 Scores_Obtained float,\
 MaxMarks int\
 )')

0


In [ ]:
connection = pymysql.connect(host=hostName,
                             user=userName,
                             password=passWord,
                             db=dbName)
cursor=connection.cursor()

for index, row in overall_scores.iterrows():
  sql = "INSERT INTO overall_Scores (userid, saved_date, Category, Scores_Obtained, MaxMarks) VALUES (%s, %s, %s, %s, %s)"

  cursor.execute(sql, (row.userid, row.saved_date, row.Category, row.Scores_Obtained, row.MaxMarks))

connection.commit()
connection.close()

In [ ]:
temp = r('select * from overall_Scores')

In [ ]:
temp[~temp.duplicated(keep='first')]

,userid,saved_date,Category,Scores_Obtained,MaxMarks
0,ashwin.kumar@praxis.ac.in,2021-10-22,"KNN, Naive Bayes",68.000,100
1,radha.rathore@praxis.ac.in,2021-10-22,"SVM, Decision Tree, Naive Bayes",92.000,150
2,geetha.joseph@praxis.ac.in,2021-10-23,"KMeans, Logistic Regression, Decision Tree, St...",176.000,200
3,yash.aswani@praxis.ac.in,2021-10-25,"Statistics, Logistic Regression, KNN, SVM",112.000,200
4,radha.rathore@praxis.ac.in,2021-11-12,Linear Regression; SVM,18.225,100
5,shahrukh.gouhar@praxis.ac.in,2021-11-12,Linear Regression,25.000,50
